<a href="https://colab.research.google.com/github/pontonkid/Images-Chat-Assistant/blob/main/back_n_forth_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q gTTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
import torch
from transformers import BitsAndBytesConfig, pipeline
import whisper
import gradio as gr
from gtts import gTTS
from PIL import Image
import re
import os
import datetime
import locale
import numpy as np
import nltk

nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Set up locale
os.environ["LANG"] = "en_US.UTF-8"
os.environ["LC_ALL"] = "en_US.UTF-8"
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [4]:
# Configuration for quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load the image-to-text model
model_id = "llava-hf/llava-1.5-7b-hf"
pipe = pipeline("image-to-text",
                model=model_id,
                model_kwargs={"quantization_config": quantization_config})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

In [5]:
# Load the whisper model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("medium", device=DEVICE)

100%|█████████████████████████████████████| 1.42G/1.42G [00:37<00:00, 40.7MiB/s]


In [7]:
# History tracking for conversation
conversation_history = []

def writehistory(text):
    """Write history to a log file."""
    tstamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    logfile = f'{tstamp}_log.txt'
    with open(logfile, 'a', encoding='utf-8') as f:
        f.write(text + '\n')

def img2txt(input_text, input_image):
    """Convert image to text using iterative prompts."""
    try:
        image = Image.open(input_image)
        if isinstance(input_text, tuple):
            input_text = input_text[0]  # Take the first element if it's a tuple

        writehistory(f"Input text: {input_text}")
        prompt = "USER: <image>\n" + input_text + "\nASSISTANT:"
        while True:
            outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})

            if outputs and outputs[0]["generated_text"]:
                match = re.search(r'ASSISTANT:\s*(.*)', outputs[0]["generated_text"])
                reply = match.group(1) if match else "No response found."
                conversation_history.append({"user": input_text, "assistant": reply})
                prompt = "USER: " + reply + "\nASSISTANT:"
                return reply  # Only return the first response for now
            else:
                return "No response generated."
    except Exception as e:
        return str(e)

In [8]:
def transcribe(audio_path):
    """Transcribe audio to text using Whisper."""
    try:
        if not audio_path:
            return '', '', None

        audio = whisper.load_audio(audio_path)
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio).to(model.device)
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)
        return result.text
    except Exception as e:
        return str(e)

In [9]:
def text_to_speech(text, file_path):
    """Convert text to speech using gTTS."""
    try:
        language = 'en'
        audioobj = gTTS(text=text, lang=language, slow=False)
        audioobj.save(file_path)
        return file_path
    except Exception as e:
        return str(e)

In [10]:
def process_inputs(audio_path, image_path):
    """Process inputs and return results."""
    try:
        if os.path.exists("Temp3.mp3"):
            os.remove("Temp3.mp3")

        speech_to_text_output = transcribe(audio_path)

        if image_path:
            chatgpt_output = img2txt(speech_to_text_output, image_path)
        else:
            chatgpt_output = "No image provided."

        processed_audio_path = text_to_speech(chatgpt_output, "Temp3.mp3")
        return speech_to_text_output, chatgpt_output, processed_audio_path
    except Exception as e:
        return str(e)


In [11]:
iface = gr.Interface(
    fn=process_inputs,
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath"),
        gr.Image(type="filepath")
    ],
    outputs=[
        gr.Textbox(label="Speech to Text"),
        gr.Textbox(label="ChatGPT Output"),
        gr.Audio("Temp3.mp3")
    ],
    title="Learn OpenAI Whisper: Image processing with Whisper and Llava",
    description="Upload an image and interact via voice input and audio response."
)

if __name__ == "__main__":
    iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cb575072b6f1f22706.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cb575072b6f1f22706.gradio.live


In [12]:
import gradio as gr
import os

# Initialize conversation history
conversation_history = []

def process_inputs(audio_path, image_path):
    """Process inputs and return results."""
    global conversation_history
    try:
        if os.path.exists("Temp3.mp3"):
            os.remove("Temp3.mp3")

        speech_to_text_output = transcribe(audio_path)

        if image_path:
            chatgpt_output = img2txt(speech_to_text_output, image_path)
        else:
            chatgpt_output = "No image provided."

        processed_audio_path = text_to_speech(chatgpt_output, "Temp3.mp3")

        # Append the latest interaction to the conversation history
        conversation_history.append({"user": speech_to_text_output, "assistant": chatgpt_output})

        # Format the conversation history for display
        chat_history_display = ""
        for chat in conversation_history:
            chat_history_display += f"User: {chat['user']}\nAssistant: {chat['assistant']}\n\n"

        return speech_to_text_output, chat_history_display, processed_audio_path

    except Exception as e:
        return str(e)

iface = gr.Interface(
    fn=process_inputs,
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath"),
        gr.Image(type="filepath")
    ],
    outputs=[
        gr.Textbox(label="Speech to Text"),
        gr.Textbox(label="Chat History"),
        gr.Audio("Temp3.mp3")
    ],
    title="Learn OpenAI Whisper: Image processing with Whisper and Llava",
    description="Upload an image and interact via voice input and audio response."
)

if __name__ == "__main__":
    iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2466f274744c29024f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2466f274744c29024f.gradio.live


In [14]:
# Global variables to keep track of conversation history
conversation_history = []


In [15]:
def transcribe(audio_path):
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text

def img2txt(input_text, input_image):
    image = Image.open(input_image)
    prompt = "USER: <image>\n" + input_text + "\nASSISTANT:"
    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
    if outputs and outputs[0]["generated_text"]:
        match = re.search(r'ASSISTANT:\s*(.*)', outputs[0]["generated_text"])
        reply = match.group(1) if match else "No response found."
        return reply
    else:
        return "No response generated."

def text_to_speech(text, file_path):
    audioobj = gTTS(text=text, lang='en', slow=False)
    audioobj.save(file_path)
    return file_path

def chatbot_interface(audio_path, image_path, user_message):
    global conversation_history

    # Step 1: Process the audio input if provided
    if audio_path is not None:
        user_message = transcribe(audio_path)

    # Step 2: Generate the assistant response
    if image_path is not None:
        assistant_message = img2txt(user_message, image_path)
    else:
        assistant_message = "No image provided."

    # Step 3: Convert the assistant message to speech
    audio_response_path = text_to_speech(assistant_message, "Temp3.mp3")

    # Step 4: Append the new messages to the conversation history
    conversation_history.append(("User", user_message))
    conversation_history.append(("Assistant", assistant_message))

    # Step 5: Format the conversation history for display
    chat_history_display = [{"role": role, "content": message} for role, message in conversation_history]

    return chat_history_display, audio_response_path

# Define the Gradio interface
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Audio(type="filepath", label="Speak or Type your message"),
        gr.Image(type="filepath", label="Upload an image"),
        gr.Textbox(lines=2, placeholder="Type your message here...", label="User message (if no audio)")
    ],
    outputs=[
        gr.Chatbot(label="Chat History"),
        gr.Audio("Temp3.mp3")
    ],
    title="Interactive Chatbot with Image and Voice Processing",
    description="Interact with the chatbot by speaking, typing, and uploading images."
)

if __name__ == "__main__":
    iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9e22dcda93dec66f2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9e22dcda93dec66f2e.gradio.live


In [18]:
def transcribe(audio_path):
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text

def img2txt(input_text, input_image):
    image = Image.open(input_image)
    prompt = "USER: <image>\n" + input_text + "\nASSISTANT:"
    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
    if outputs and outputs[0]["generated_text"]:
        match = re.search(r'ASSISTANT:\s*(.*)', outputs[0]["generated_text"])
        reply = match.group(1) if match else "No response found."
        return reply
    else:
        return "No response generated."

def text_to_speech(text, file_path):
    audioobj = gTTS(text=text, lang='en', slow=False)
    audioobj.save(file_path)
    return file_path

def chatbot_interface(audio_path, image_path, user_message):
    global conversation_history

    # Step 1: Process the audio input if provided
    if audio_path:
        user_message = transcribe(audio_path)

    # Step 2: Generate the assistant response
    if image_path:
        assistant_message = img2txt(user_message, image_path)
    else:
        assistant_message = "No image provided."

    # Step 3: Convert the assistant message to speech
    audio_response_path = text_to_speech(assistant_message, "Temp3.mp3")

    # Step 4: Append the new messages to the conversation history
    conversation_history.append(("User", user_message))
    conversation_history.append(("Assistant", assistant_message))

    # Step 5: Format the conversation history for display
    chat_history_display = [{"role": role, "content": message} for role, message in conversation_history]

    return chat_history_display, audio_response_path

# Define the Gradio interface
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Audio(type="filepath", label="Record your message"),
        gr.Image(type="filepath", label="Upload an image"),
        gr.Textbox(lines=2, placeholder="Type your message here...", label="User message (if no audio)")
    ],
    outputs=[
        gr.Chatbot(label="Chat History"),
        gr.Audio(label="Assistant Response")
    ],
    title="Interactive Chatbot with Image and Voice Processing",
    description="Interact with the chatbot by recording, typing, and uploading images."
)

if __name__ == "__main__":
    iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5cab830f4509d4157f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1907, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1730, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py", line 487, in postprocess
    self._check_format(value, "tuples")
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py", line 270, in _check_format
    raise Error(
gradio.exceptions.Error: 'Data incompatible with tuples format. Each me

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5cab830f4509d4157f.gradio.live


In [19]:
# Global variables to keep track of conversation history
conversation_history = []


In [21]:
def transcribe(audio_path):
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text

def img2txt(input_text, input_image):
    image = Image.open(input_image)
    prompt = "USER: <image>\n" + input_text + "\nASSISTANT:"
    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
    if outputs and outputs[0]["generated_text"]:
        match = re.search(r'ASSISTANT:\s*(.*)', outputs[0]["generated_text"])
        reply = match.group(1) if match else "No response found."
        return reply
    else:
        return "No response generated."

def text_to_speech(text, file_path):
    audioobj = gTTS(text=text, lang='en', slow=False)
    audioobj.save(file_path)
    return file_path

def chatbot_interface(audio_path, image_path, user_message):
    global conversation_history

    # Step 1: Process the audio input if provided
    if audio_path:
        user_message = transcribe(audio_path)

    # Step 2: Generate the assistant response
    if image_path:
        assistant_message = img2txt(user_message, image_path)
    else:
        assistant_message = "No image provided."

    # Step 3: Convert the assistant message to speech
    audio_response_path = text_to_speech(assistant_message, "Temp3.mp3")

    # Step 4: Append the new messages to the conversation history
    conversation_history.append(("user", user_message))
    conversation_history.append(("assistant", assistant_message))

    # Step 5: Format the conversation history for display
    chat_history_display = [(role, message) for role, message in conversation_history]

    return chat_history_display, audio_response_path

# Define the Gradio interface
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Audio(type="filepath", label="Record your message"),
        gr.Image(type="filepath", label="Upload an image"),
        gr.Textbox(lines=2, placeholder="Type your message here...", label="User message (if no audio)")
    ],
    outputs=[
        gr.Chatbot(label="Chat History"),
        gr.Audio(label="Assistant Response")
    ],
    title="Interactive Chatbot with Image and Voice Processing",
    description="Interact with the chatbot by recording, typing, and uploading images."
)

if __name__ == "__main__":
    iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2d5ffa37c5fbdc0a04.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2d5ffa37c5fbdc0a04.gradio.live
